# Feature selection with Rough set theory

In [23]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy.stats import norm
from sklearn.preprocessing import StandardScaler
from scipy import stats
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [24]:
DF = pd.read_csv('Downloads\Flu_dataset.csv')

In [25]:
DF.head(6)

,Headache,Muscle-pain,Temperature,Flu
0,Yes,Yes,very high,Yes
1,Yes,No,high,Yes
2,Yes,No,high,No
3,No,Yes,normal,No
4,No,Yes,high,Yes
5,No,Yes,very high,Yes


In [5]:
#fonction qui permet de retourner une listes sans des doublons 
def unique_set(my_list): 
    unique_sets = []
    for s in my_list:
        if s not in unique_sets:
            unique_sets.append(s)
    return unique_sets

In [6]:
# fonction qui permet de retourner un dict avec comme clé la position dans le dict de base et comme valeur la valeur du dict de base 
def reindex_dict(d):
    new_dict = {}
    for i, (key, value) in enumerate(d.items()):
        new_dict[i] = value
    return new_dict

In [33]:
#fonction qui permet de créer les classes d'objet ayant les memes valeurs 
def IND_classes_short(iterable, indices):
    classes = {}
    dclasses = {}
    #indices c'est les indices des attributs dans le dataset : cas 1 : si c B alors c tous les features (combinaisons possibles)
    # cas 2 : si c le D c la target ducoup un seul indice
    #iterable c l'univers U en gros c les lignes du dataset 
    #print("les indices : ",indices,"\n")
    for i, o in enumerate(iterable):
        #print("l'objet ",i," : ",o,"\n")
        #print(" o de indice : ",tuple(o[ind] for ind in indices))
        key = tuple(o[ind] for ind in indices)
        #print("key : ", key,"\n")
        if key not in classes:
            classes[key] = set()
        classes[key].add(i)
        #print("classes(key) : ",classes[key],"\n")
        #print("dclasses(",i,") : \n")
        dclasses[i] = classes[key]
    #print("les classes : classes : ",reindex_dict(classes))  <<<<<<<<<<<<<<<<<<<<
    
    print("les classes : dclasses : ",dclasses)
    #return list(classes.values()), dclasses
    return reindex_dict(classes),dclasses

In [31]:
#la lower approx utilisé dans RST 
def lower_2(B,D):
    subset_list = []
    for key1, value1 in D.items():
        for key2, value2 in B.items():
            if value2.issubset(value1):
                #print("\nles objets d'une classe de D : ",value1,"  et celle de B : ", value2,"\n")
                subset_list.append(value2)
    print("la liste d'approximation lower2 : ",unique_set(subset_list),"\n\n")
    return subset_list

In [9]:
def upper_2(B,D):
    subset_list = []
    for key1, value1 in D.items():
        for key2, value2 in B.items():
            if value2.intersection(value1):
                #print("\nles objets d'une classe de D : ",value1,"  et celle de B : ", value2,"\n")
                subset_list.append(value2)
    print("la liste d'approximation upper2 : ",unique_set(subset_list),"\n\n")
    return subset_list


In [10]:
def boundary_region(list1, list2):
    #print("DANS BOUNARY  voila la liste : ",list1,"\n")
    if list1 != None :
        if list2 == None : 
              return set(list1)
        
        #set1 = set(list1)
        #set2 = set(list2)
        #result = set()
        result = list()
        for s1 in list1:
            if s1 not in list2:
                #result.add(s1)
                result.append(s1)
        return unique_set(result)
    return set()

In [11]:
#fonction qui permet de faire une comparaison entre deux listes avec elements simples
def compare_lists(list1, list2):
    # Print the lists
    #print("List 1: ", list1)
    #print("List 2: ", list2)
    
    # Compare the lists
    if set(list1) == set(list2):
        return True
    else:
        return False

In [12]:
#fonction qui permet de retourner une liste et retourne l'équivalent en dict avec comme clé les indices des elements
def feature_index_dict(feature_list):
    index_dict = {}
    for i, feature in enumerate(feature_list):
        index_dict[i] = feature
    return index_dict

In [13]:
# fonction qui retourner une list avec les features indiqué avec la liste des indices en parametres
def index_to_feature(index_list, df):
    feature_list = []
    for i in index_list :
        feature_list.append(df.columns[i])
    return feature_list


# comment génerer les combinaisons des différents attributs

In [26]:
import itertools

def generate_combinations(attributes):
    """
  Generate all possible combinations of attributes, with a minimum of 2 attributes and a maximum of all attributes.

  Parameters:
  - attributes: a list of attribute names

  Returns:
  - combinations: a list of lists, each representing a combination of attribute names
  """
  # Initialize an empty list to store the combinations
    combinations = []

  # Iterate through all possible lengths of combinations
    for length in range(1, len(attributes)+1):
    # Generate all combinations of the given length using itertools.combinations
        for combination in itertools.combinations(attributes, length):
            # Add the combination to the list of combinations
            combinations.append(list(combination))

    # Return the list of combinations
    return combinations

In [20]:
U = DF.values.tolist()
U = [[index] + value for index , value in enumerate(U) ]

#_________________________ Finding lower approximation and positif region
def positive_reg(B,D):
    #ind_B = equivalence_partition( U , B )[1]
    tmp_1 = IND_classes_short( U , B )
    ind_B = tmp_1[1] 
    #print(IND_classes_short(U,B))
    #print("index B : ",ind_B,"\n")
    #ind_d = equivalence_partition( U , D )[1]
    tmp_2 = IND_classes_short( U , D )
    ind_d = tmp_2[1]
    #print("index D : ",ind_d,"\n")
    print("les features : ",B,"\n")
    
    #lower_2(tmp_1[0],tmp_2[0])
    #upper_2(tmp_1[0],tmp_2[0])
    lower_res = lower_2(tmp_1[0],tmp_2[0])
    print("la boundary region : ",boundary_region( upper_2(tmp_1[0],tmp_2[0]) , lower_res ) ,"\n\n")
    
    return lower_res
#_________________________ Rough set feature selection algo <<<<<<<<<<<<<<<<<<<<<<<<<<<< A REFAIRE 

def RST(C,D):
    
    les_positives = list()
    positive_all_features = [item for sublist in positive_reg(C,D) for item in sublist]
    #positive_all_features = positive_reg(C,D)
    dict_features = feature_index_dict(DF.columns)
    # parcourir les combinaisons
    combs = generate_combinations(C)
    co = 0
    #print("La combi 12 : \n",index_to_feature(combs[12],DF))
    
    
    for elem in combs : 
        print(elem,"\n\n")
        print(index_to_feature(elem,DF))
        #tmp_positive_reg = positive_reg(elem,D)
        tmp_positive_reg = [item for sublist in positive_reg(elem,D) for item in sublist]
        if (compare_lists(positive_all_features,tmp_positive_reg)) : 
            #print("Pos en generale : ",positive_all_features,"\nla Pos d'une combi",tmp_positive_reg,"\n")
            co = co + 1 
        #if compare_lists(positive_all_features,tmp_positive_reg) : 
            les_positives.append(elem)
    print ("le nombre de reducts co :" , co)
    
    return les_positives
    # récupérer pour chaque combinaison la positive région

In [34]:

# Define the indices of the feature columns
feature_indices = range(1,len(DF.columns))

# Set D to the decision index
D = [len(DF.columns)]

# Set B to the set of feature indices
B = set(range(1,len(DF.columns[:-1])))

resultat = RST(B,D)

les classes : dclasses :  {0: {0}, 1: {1, 2}, 2: {1, 2}, 3: {3, 4, 5}, 4: {3, 4, 5}, 5: {3, 4, 5}}
les classes : dclasses :  {0: {0, 1, 4, 5}, 1: {0, 1, 4, 5}, 2: {2, 3}, 3: {2, 3}, 4: {0, 1, 4, 5}, 5: {0, 1, 4, 5}}
les features :  {1, 2} 

la liste d'approximation lower2 :  [{0}] 


la liste d'approximation upper2 :  [{0}, {1, 2}, {3, 4, 5}] 


la boundary region :  [{1, 2}, {3, 4, 5}] 


[1] 


['Muscle-pain']
les classes : dclasses :  {0: {0, 1, 2}, 1: {0, 1, 2}, 2: {0, 1, 2}, 3: {3, 4, 5}, 4: {3, 4, 5}, 5: {3, 4, 5}}
les classes : dclasses :  {0: {0, 1, 4, 5}, 1: {0, 1, 4, 5}, 2: {2, 3}, 3: {2, 3}, 4: {0, 1, 4, 5}, 5: {0, 1, 4, 5}}
les features :  [1] 

la liste d'approximation lower2 :  [] 


la liste d'approximation upper2 :  [{0, 1, 2}, {3, 4, 5}] 


la boundary region :  [{0, 1, 2}, {3, 4, 5}] 


[2] 


['Temperature']
les classes : dclasses :  {0: {0, 3, 4, 5}, 1: {1, 2}, 2: {1, 2}, 3: {0, 3, 4, 5}, 4: {0, 3, 4, 5}, 5: {0, 3, 4, 5}}
les classes : dclasses :  {0: {0, 1, 4, 5}, 

In [28]:
resultat

[[1, 2]]